# Análise de Inteligência Comercial - FMCG Doces e Guloseimas
## ETL - Extração, Transformação e Carregamento de Dados
Neste notebook, vamos executar as etapas de ETL nas bases **Sell In** e **Sell Out**, preparando-as para análise exploratória e modelagem.
---

In [1]:
import pandas as pd
import numpy as np
from datetime import datetime

# Mapeamento para parsing de mês abreviado
mes_map = {
    'jan': 1, 'fev': 2, 'mar': 3, 'abr': 4,
    'mai': 5, 'jun': 6, 'jul': 7, 'ago': 8,
    'set': 9, 'out': 10, 'nov': 11, 'dez': 12
}

def parse_mes_ano(x):
    s = str(x).strip().lower()
    # Formato MM/YYYY
    if '/' in s:
        try:
            return datetime.strptime(s, '%m/%Y')
        except:
            pass
    # Formato 'abr 2023'
    parts = s.split()
    if len(parts) == 2:
        mes_abbr, ano = parts
        mes = mes_map.get(mes_abbr[:3])
        if mes:
            return datetime(int(ano), mes, 1)
    raise ValueError(f'Formato de data não reconhecido: {x!r}')

### 1. ETL da base **Sell In**

In [2]:
# 1.1 Carrega o arquivo
sell_in = pd.read_excel(r'C:\Users\frdsz\Downloads\Case Fini v0.1\data\raw\base sell in.xlsx')

# 1.2 Limpeza nos nomes de colunas
sell_in.columns = (sell_in.columns
    .str.strip()
    .str.replace(r'[\r\n\t]+', ' ', regex=True)
)

# 1.3 Parse de datas e criação de colunas auxiliares
sell_in['Data'] = sell_in['Calendario[Mês/Ano]'].apply(parse_mes_ano)
sell_in['Ano_Mes'] = sell_in['Data'].dt.strftime('%Y-%m')

# 1.4 Conversão de colunas numéricas
sell_in['[SumP_NF_ValTotal]'] = (sell_in['[SumP_NF_ValTotal]']
    .replace(['-', ''], np.nan)
    .astype(float)
)
sell_in['[SumPeso_Liquido]'] = (sell_in['[SumPeso_Liquido]']
    .replace(['-', ''], np.nan)
    .astype(float)
)
sell_in['[SumP_NF_Quantidade]'] = (
    pd.to_numeric(sell_in['[SumP_NF_Quantidade]'], errors='coerce')
    .fillna(0).astype(int)
)

# 1.5 Filtragem de vendas válidas
sell_in = sell_in[
    (sell_in['FATO[P.NF.OperacaoFini]'] == 'Venda') &
    (~sell_in['Produto[P.Produtos.LinhaProducao]']
       .str.contains('PROMOCIONAIS', case=False, na=False)) &
    (sell_in['[SumP_NF_ValTotal]'].notna())
].copy()

# 1.6 Correção de tipo de produto para MARSH
mask = (
    (sell_in['Produto[P.Produtos.LinhaProducao]'] == 'MARSHMALLOW') &
    sell_in['Produto[P.Produtos.TpPro.Inteligente]'].fillna('').eq('')
)
sell_in.loc[mask, 'Produto[P.Produtos.TpPro.Inteligente]'] = 'MARSH'

# 1.7 Renomeação de colunas para conveniência
sell_in.rename(columns={
    'Calendario[Mês/Ano]': 'Origem_Calendario',
    'Data': 'Data_SellIn',
    'FATO[P.NF.Ramo_Atividade]': 'Ramo_Atividade',
    'Cliente[P.Cliente.Pais]': 'Pais',
    'Cliente[P.Cliente.Regiao]': 'Regiao',
    'FATO[P.NF.OperacaoFini]': 'Tipo_Operacao',
    'Cliente[P.Cliente.NomeFantasia]': 'Cliente',
    'Cliente[P.Cliente.UF]': 'UF',
    'Produto[P.Produtos.LinhaProducao]': 'Linha_Producao',
    'Produto[P.Produtos.Gramagem]': 'Gramagem',
    'Produto[P.Produtos.MacroFormato]': 'Macro_Formato',
    'Produto[P.Produtos.Nome]': 'Nome_Produto',
    'Produto[P.Produtos.Grupo]': 'Grupo_Produto',
    'Produto[P.Produtos.TpPro.Inteligente]': 'Tipo_Produto',
    'Cliente[P.Cliente.Municipio]': 'Municipio',
    'Hierarquia[P.Cliente.Canal_Hierarquia]': 'Canal',
    'Hierarquia[P.Cliente.GerenteRegional]': 'Gerente_Regional',
    'Hierarquia[P.Cliente.VendedorNome]': 'Vendedor',
    'Produto[Cod. Produto]': 'Cod_Produto',
    '[SumP_NF_Quantidade]': 'Quantidade',
    '[SumP_NF_ValTotal]': 'Valor_Total',
    '[SumPeso_Liquido]': 'Peso_Liquido',
}, inplace=True)

# 1.8 Inspeção rápida
print(sell_in.info())
sell_in.head()

<class 'pandas.core.frame.DataFrame'>
Index: 4172 entries, 0 to 13305
Data columns (total 24 columns):
 #   Column             Non-Null Count  Dtype         
---  ------             --------------  -----         
 0   Origem_Calendario  4172 non-null   object        
 1   Ramo_Atividade     4172 non-null   object        
 2   Pais               4172 non-null   object        
 3   Regiao             4172 non-null   object        
 4   Tipo_Operacao      4172 non-null   object        
 5   Cliente            4172 non-null   object        
 6   UF                 4172 non-null   object        
 7   Linha_Producao     4172 non-null   object        
 8   Gramagem           4172 non-null   object        
 9   Macro_Formato      4172 non-null   object        
 10  Nome_Produto       4172 non-null   object        
 11  Grupo_Produto      4172 non-null   object        
 12  Tipo_Produto       4172 non-null   object        
 13  Municipio          4172 non-null   object        
 14  Canal       

C:\Users\frdsz\AppData\Local\Temp\ipykernel_17812\1104002963.py:16: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  .replace(['-', ''], np.nan)
C:\Users\frdsz\AppData\Local\Temp\ipykernel_17812\1104002963.py:20: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  .replace(['-', ''], np.nan)


,Origem_Calendario,Ramo_Atividade,Pais,Regiao,Tipo_Operacao,Cliente,UF,Linha_Producao,Gramagem,Macro_Formato,...,Canal,Gerente_Regional,Vendedor,Cod_Produto,Quantidade,Quantidade,Valor_Total,Peso_Liquido,Data_SellIn,Ano_Mes
0,abr 2023,DISTRIBUIDORES,BRASIL,SUDESTE,Venda,ONTARGET,SP,PASTILHAS,14 G,MONOPORÇAO,...,DISTRIBUIDORES,TERRITORIO SP,DISTR ONTARGET - SP,110014F08501,CX,40,6891.99,268.8,2023-04-01,2023-04
1,abr 2023,DISTRIBUIDORES,BRASIL,SUL,Venda,CBN DISTRIBUIDORA,PR,PASTILHAS,14 G,MONOPORÇAO,...,DISTRIBUIDORES,TERRITORIO SUL,DIST CBN,110014F08501,CX,60,13141.80,403.2,2023-04-01,2023-04
2,abr 2023,DISTRIBUIDORES,BRASIL,SUL,Venda,CBN DISTRIBUIDORA,PR,PASTILHAS,14 G,MONOPORÇAO,...,DISTRIBUIDORES,TERRITORIO SUL,DIST CBN,110014F08501,CX,60,13141.80,403.2,2023-04-01,2023-04
9,abr 2023,DISTRIBUIDORES,BRASIL,SUL,Venda,CBN DISTRIBUIDORA,PR,PASTILHAS,14 G,MONOPORÇAO,...,DISTRIBUIDORES,TERRITORIO SUL,DIST CBN,110014F08601,CX,30,6570.90,201.6,2023-04-01,2023-04
10,abr 2023,DISTRIBUIDORES,BRASIL,SUL,Venda,CBN DISTRIBUIDORA,PR,PASTILHAS,14 G,MONOPORÇAO,...,DISTRIBUIDORES,TERRITORIO SUL,DIST CBN,110014F08601,CX,30,6570.90,201.6,2023-04-01,2023-04


### 2. ETL da base **Sell Out**

In [3]:
# 2.1 Carrega o arquivo
sell_out = pd.read_excel(r'C:\Users\frdsz\Downloads\Case Fini v0.1\data\raw\BASE SELL OUT.xlsx')

# 2.2 Limpeza nos nomes de colunas
sell_out.columns = (sell_out.columns
    .str.strip()
    .str.replace(r'[\r\n\t]+', ' ', regex=True)
)

# 2.3 Conversão de datas
sell_out['Data'] = pd.to_datetime(
    sell_out['Ano/Mês'], errors='raise')
sell_out['Ano_Mes'] = sell_out['Data'].dt.strftime('%Y-%m')

# 2.4 Conversão de colunas numéricas
for col in ['Sell through valor', 'Sell through cx']:
    sell_out[col] = pd.to_numeric(
        sell_out[col].astype(str)
            .str.replace(r'[^0-9,.-]', '', regex=True)
            .str.replace(',', '.', regex=False)
            .replace('', np.nan),
        errors='coerce'
    )
sell_out['Sell through unidade'] = (
    pd.to_numeric(sell_out['Sell through unidade'], errors='coerce')
    .fillna(0).astype(int)
)

# 2.5 Filtragem de valores positivos
sell_out = sell_out[sell_out['Sell through valor'] > 0].copy()

# 2.6 Renomeação de colunas
sell_out.rename(columns={
    'Ano/Mês': 'Origem_AnoMes',
    'Agente de Distribuição': 'Agente_Distribuicao',
    'UF PDV': 'UF_PDV',
    'Cidade PDV': 'Cidade_PDV',
    'Segmentação Mtrix': 'Segmentacao_Mtrix',
    'Cód Produto (SKU)': 'Cod_Produto',
    'Nome Produto': 'Nome_Produto',
    'Sell through valor': 'Valor_SellThrough',
    'Sell through unidade': 'Unidades_SellThrough',
    'Sell through cx': 'Caixas_SellThrough',
}, inplace=True)

# 2.7 Inspeção rápida
print(sell_out.info())
sell_out.head()

<class 'pandas.core.frame.DataFrame'>
Index: 25708 entries, 0 to 77275
Data columns (total 12 columns):
 #   Column                Non-Null Count  Dtype         
---  ------                --------------  -----         
 0   Origem_AnoMes         25708 non-null  datetime64[ns]
 1   Agente_Distribuicao   25708 non-null  object        
 2   UF_PDV                25708 non-null  object        
 3   Cidade_PDV            25708 non-null  object        
 4   Segmentacao_Mtrix     25708 non-null  object        
 5   Cod_Produto           25708 non-null  object        
 6   Nome_Produto          25708 non-null  object        
 7   Valor_SellThrough     25708 non-null  float64       
 8   Unidades_SellThrough  25708 non-null  int32         
 9   Caixas_SellThrough    25708 non-null  float64       
 10  Data                  25708 non-null  datetime64[ns]
 11  Ano_Mes               25708 non-null  object        
dtypes: datetime64[ns](2), float64(2), int32(1), object(7)
memory usage: 2.5+ MB
Non

,Origem_AnoMes,Agente_Distribuicao,UF_PDV,Cidade_PDV,Segmentacao_Mtrix,Cod_Produto,Nome_Produto,Valor_SellThrough,Unidades_SellThrough,Caixas_SellThrough,Data,Ano_Mes
0,2023-11-01,ONTARGET,SP,MOGI DAS CRUZES,LOJAS DE CONVENIENCIA,111080C01506,CHICLE MELANCIA 12X80G,28.92,6,0.5,2023-11-01,2023-11
3,2023-11-01,ONTARGET,SP,MOGI DAS CRUZES,LOJAS DE CONVENIENCIA,111080C01606,CHICLE OVOS DINOSSAURO 12X80G,28.92,6,0.5,2023-11-01,2023-11
6,2023-11-01,ONTARGET,SP,MOGI DAS CRUZES,LOJAS DE CONVENIENCIA,111080C01706,CHICLE SALADA FRUTAS 12X80G,28.92,6,0.5,2023-11-01,2023-11
9,2023-11-01,ONTARGET,SP,MOGI DAS CRUZES,LOJAS DE CONVENIENCIA,111100A04003,AROS DE MORANGO 12X80G,28.92,6,0.5,2023-11-01,2023-11
12,2023-11-01,ONTARGET,SP,MOGI DAS CRUZES,LOJAS DE CONVENIENCIA,111100M12002,MINHOCAS BRILHO 12X80G,28.92,6,0.5,2023-11-01,2023-11


### 3. Salvando bases tratadas

In [4]:
from pathlib import Path

# 1. Define o diretório de saída, relativo ao notebook em notebooks/
project_root = Path.cwd().parent             # sobe 1 nível (de notebooks/ até project_case/)
processed_dir = project_root / "data" / "processed"

# 2. Garante que data/processed existe
processed_dir.mkdir(parents=True, exist_ok=True)

# 3. Exporta os CSVs para lá
sell_in.to_csv(processed_dir / "sell_in_processed.csv", index=False)
sell_out.to_csv(processed_dir / "sell_out_processed.csv", index=False)

print(f"Bases tratadas salvas em: {processed_dir.resolve()}")


Bases tratadas salvas em: C:\Users\frdsz\Downloads\Case Fini v0.1\data\processed
